In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from helpers import load_distilbert
import json
from sacred import Experiment
from sacred.observers import MongoObserver, FileStorageObserver

import numpy as np
from baselines import ZeroBaselineFactory
from evaluators import ProportionalityEvaluator
from helpers import load_albert_v2, load_imdb_albert_lig_data, extract_token_ids_and_attributions, load_distilbert
from attribution_methods import RandomAttributionValues, HillClimber
from tqdm import tqdm

# hill climbing search of a one-dimensional objective function
from numpy import asarray
from numpy import arange
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed
from matplotlib import pyplot
from evaluators import ProportionalityEvaluator

In [14]:
data = load_imdb_albert_lig_data()
model = load_distilbert(from_notebook=1)

Reusing dataset thermostat (/home/tim/.cache/huggingface/datasets/thermostat/imdb-albert-lig/1.0.1/0cbe93e1fbe5b8ed0217559442d8b49a80fd4c2787185f2d7940817c67d8707b)


Loading Thermostat configuration: imdb-albert-lig


In [15]:
observations, lig_attributions = zip(*[extract_token_ids_and_attributions(d) for d in data[:200]])
del data

evaluator = ProportionalityEvaluator(model=model, baseline_factory=ZeroBaselineFactory)
tpn_objective = lambda x: evaluator.compute_tpn(observation=observation, attribution_values=x)
tps_objective = lambda x: evaluator.compute_tps(observation=observation, attribution_values=x)


In [16]:
hill_climber_tpn = HillClimber(objective=tpn_objective, bounds=(0,1), iterations=60, step_size=.1)
hill_climber_tps = HillClimber(objective=tps_objective, bounds=(0,1), iterations=60, step_size=.1)

## Running Hillclimbing once on all samples

In [7]:
tpn_results, tps_results = [], []
for i in tqdm(range(len(observations))):
    observation = observations[i]
    hc_tpn_results,_,_ = hill_climber_tpn.get_attribution_values(observation=observation)
    tpn_results.append(evaluator.compute_tpn(observation=observation, attribution_values=hc_tpn_results))
    tps_results.append(evaluator.compute_tps(observation=observation,
											 attribution_values=hc_tpn_results))

  0%|          | 0/2 [00:00<?, ?it/s]

>3 = 0.02599
>62 = 0.02584
>65 = 0.02504
>70 = 0.02411
>72 = 0.01939
>74 = 0.01910
>76 = 0.01899
>86 = 0.01881
>97 = 0.01873
>98 = 0.01560


 50%|█████     | 1/2 [00:55<00:55, 55.24s/it]

>2 = 0.00651


100%|██████████| 2/2 [04:12<00:00, 126.35s/it]


In [8]:
with open("../data/tpn_hillclimber_all_samples.json","w") as fp:
    json.dump(dict(tpn_results=tpn_results, tps_results=tps_results), fp)


In [ ]:
tpn_results, tps_results = [], []
for i in tqdm(range(len(observations))):
    observation = observations[i]
    hc_tps_results,_,_ = hill_climber_tps.get_attribution_values(observation=observation)
    tpn_results.append(evaluator.compute_tpn(observation=observation,
                                             attribution_values=hc_tps_results))
    tps_results.append(evaluator.compute_tps(observation=observation,
											 attribution_values=hc_tps_results))

In [ ]:
with open("../data/tps_hillclimber_all_samples.json","w") as fp:
    json.dump(dict(tpn_results=tpn_results, tps_results=tps_results), fp)

## Running Hillclimbing repeatedly on one sample

In [17]:
iterations = 200
random_idx = 42
observation = observations[random_idx]

hc_tpn_results, hc_tps_results = [], []
for _ in range(iterations):
    hc_tps_result,_,_ = hill_climber_tpn.get_attribution_values(observation=observation)
    hc_tpn_result,_,_ = hill_climber_tpn.get_attribution_values(observation=observation)
    hc_tpn_results.append(hc_tpn_results)
    hc_tps_results.append(hc_tps_result)

>0 = 0.03566
>1 = 0.03230
>2 = 0.02672
>3 = 0.02027
>4 = 0.01559
>6 = 0.00980
>12 = 0.00908
>32 = 0.00455
>41 = 0.00284
>6 = 0.00909


KeyboardInterrupt: 

In [ ]:
with open("../data/tpn_hillclimber_one_sample.json","w") as fp:
    json.dump(dict(tpn_results=hc_tpn_results, random_tps_results=hc_tps_results), fp)

In [ ]:
iterations = 2
random_idx = 42
observation = observations[random_idx]

hc_tpn_results, hc_tps_results = [], []
for _ in range(iterations):
    hc_tps_result,_,_ = hill_climber_tps.get_attribution_values(observation=observation)
    hc_tpn_result,_,_ = hill_climber_tps.get_attribution_values(observation=observation)
    hc_tpn_results.append(hc_tpn_results)
    hc_tps_results.append(hc_tps_result)

>0 = 0.02065
>1 = 0.01795
>6 = 0.01014
>9 = 0.00933
>13 = 0.00660
>19 = 0.00517
>21 = 0.00446
>42 = 0.00357
>44 = 0.00176
>0 = 0.01924
>3 = 0.00657
>8 = 0.00598
>21 = 0.00564


In [ ]:
with open("../data/tps_hillclimber_one_sample.json","w") as fp:
    json.dump(dict(tpn_results=hc_tpn_results, random_tps_results=hc_tps_results), fp)